In [ ]:
import sys
sys.path.append('..')
import umbrella_mesh, umbrella_optimization
import elastic_rods
import linkage_vis
from umbrella_mesh import UmbrellaEnergyType
from bending_validation import suppress_stdout as so
from visualization_helper import *
import numpy as np

### Initial Deployment

In [ ]:
input_path = '../../data/{}.json'.format('hemisphere')
target_mesh_path = '../../data/target_meshes/{}.obj'.format('hemisphere')

In [ ]:
from helpers.deployment_helper import get_deployed_umbrella_mesh

In [ ]:
from load_jsondata import read_data
input_data, io = read_data(filepath = input_path)

In [ ]:
curr_um = umbrella_mesh.UmbrellaMesh(target_mesh_path, io)

In [ ]:
curr_um.energy(UmbrellaEnergyType.Full)

#### Pin Rigid Motion



In [ ]:
fixedVars = curr_um.rigidJointAngleDoFIndices()

In [ ]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 1
OPTS.beta = 1e-6
OPTS.niter = 600
OPTS.verboseNonPosDef = False

In [ ]:
curr_um = umbrella_mesh.UmbrellaMesh(io)
thickness = io.material_params[6]
curr_um.targetDeploymentHeight = thickness

In [ ]:
curr_um.uniformDeploymentEnergyWeight = 1e-3
# curr_um.deploymentForceType = umbrella_mesh.DeploymentForceType.Constant
curr_um.targetDeploymentHeight = thickness * 5
curr_um.repulsionEnergyWeight = 0
curr_um.attractionWeight = 1
curr_um.setHoldClosestPointsFixed(False)
curr_um.scaleInputPosWeights(0.5)

In [ ]:
curr_um.energyElastic(), curr_um.energyDeployment(), curr_um.energyRepulsion(), curr_um.energyAttraction()

In [ ]:
dof = curr_um.getDoFs()
for i in range(curr_um.numJoints()):
    if (curr_um.joint(i).jointType() == umbrella_mesh.JointType.X):
        dof[curr_um.dofOffsetForJoint(i) + 6] = 1e-3
curr_um.setDoFs(dof)

In [ ]:
angles = []
def eqm_callback(prob, i):
    angles.append(curr_um.getDoFs()[curr_um.jointAngleDoFIndices()])

In [ ]:
results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)

In [ ]:
results.success

### Initialize Design Optimization

In [ ]:
import umbrella_optimization
import umbrella_optimization_finite_diff

In [ ]:
EO = umbrella_optimization.ElasticEnergyObjective(curr_um)

In [ ]:
x0 = curr_um.getDoFs().copy()

In [ ]:
optimizer = umbrella_optimization.UmbrellaOptimization(curr_um, OPTS, 2.5, -1, False, fixedVars)

In [ ]:
optimizer.setHoldClosestPointsFixed(True, True)

In [ ]:
optimizer.setAttractionWeight(optimizer.getAttractionWeight())

In [ ]:
np.linalg.norm(curr_um.getDoFs() - x0) / np.linalg.norm(x0)

In [ ]:
def set_weight_for_one_type(optimizer, curr_type):
    optimizer.beta = 0
    optimizer.gamma = 0
    if curr_type == umbrella_optimization.OptEnergyType.Full:
        optimizer.beta = 5e5
        optimizer.gamma = 1
    elif curr_type == umbrella_optimization.OptEnergyType.Target:
        optimizer.beta = 5e5
    elif curr_type == umbrella_optimization.OptEnergyType.Umbrella:
        optimizer.gamma = 1

In [ ]:
energyTerm = optimizer.objective.terms[0].term
tgtfitTerm = optimizer.objective.terms[1].term
stressTerm = optimizer.objective.terms[-1].term

In [ ]:
x = curr_um.getExtendedDoFsPARL()

In [ ]:
import fd_validation
def filterGrad(g):
    #g[curr_um.numDoF()] = 0
    return g
class ObjectiveTermFDWrapper():
    def __init__(self, term): self.term, self.obj = term, term.object()
    def numVars(self):    return self.obj.numExtendedDoFPARL()
    def getVars(self):    return self.obj.getExtendedDoFsPARL()
    def setVars(self, x): self.obj.setExtendedDoFsPARL(x); self.obj.updateSourceFrame() # Note: the source frame is always up-to-date when the design objective/gradient/hessvec is evaluated.
    def energy(self):     return self.term.value()
    def gradient(self):   return filterGrad(self.term.computeGrad())
    def hessVec(self, v): return filterGrad(self.term.computeDeltaGrad(v))

In [ ]:
import benchmark
benchmark.reset()
fd_validation.gradConvergencePlot(ObjectiveTermFDWrapper(stressTerm))
benchmark.report()

In [ ]:
import benchmark
benchmark.reset()
fd_validation.hessConvergencePlot(ObjectiveTermFDWrapper(stressTerm), testHessVec=True)
benchmark.report()

### Gradient Finite Difference Error

In [ ]:
direction = np.random.uniform(0, 1, optimizer.numParams())
direction /= np.linalg.norm(direction)

In [ ]:
optimizer.prediction_order = 1

In [ ]:
# Note: when rotation parameterization updates are enabled the rotation variable entires of "delta_x" will
# not match their finite difference counterparts (which are always zero)
optimizer.linesearchObject.disableRotationParametrizationUpdates = False
optimizer. committedObject.disableRotationParametrizationUpdates = False

In [ ]:
p = optimizer.params()

In [ ]:
delta_dJ = optimizer.apply_hess_J(p, direction)

In [ ]:
x = optimizer.linesearchObject.getDoFs().copy()
w = optimizer.w.copy()
delta_w = optimizer.delta_w.copy()
delta_x = optimizer.delta_x.copy()
d3E_w = optimizer.d3E_w
delta_w_rhs = optimizer.delta_w_rhs
H_delta_w = optimizer.get_H_times(delta_w)

In [ ]:
eps = 1e-1
J_plus = optimizer.      J(p + eps * direction)
g_plus = optimizer.gradp_J(p + eps * direction)
x_plus = optimizer.linesearchObject.getDoFs().copy()
w_plus = optimizer.w.copy()
Hw_plus = optimizer.get_H_times(w)
w_rhs_plus = optimizer.w_rhs

J_minus = optimizer.      J(p - eps * direction)
g_minus = optimizer.gradp_J(p - eps * direction)
x_minus = optimizer.linesearchObject.getDoFs().copy()
w_minus = optimizer.w.copy()
Hw_minus = optimizer.get_H_times(w)
w_rhs_minus = optimizer.w_rhs
#benchmark.report()


fd_delta_w = (w_plus - w_minus) / (2 * eps)
fd_delta_x = (x_plus - x_minus) / (2 * eps)
fd_w_rhs   = (w_rhs_plus - w_rhs_minus) / (2 * eps)
fd_Hw      = (Hw_plus - Hw_minus) / (2 * eps)

print(fd_delta_w)
print(delta_w)
print(w_plus)
print(w_minus)

In [ ]:
optimizer.J(p)
H_fd_delta_w = optimizer.get_H_times(fd_delta_w)

In [ ]:
np.linalg.norm(delta_w - fd_delta_w) / np.linalg.norm(delta_w)

In [ ]:
np.linalg.norm(delta_x - fd_delta_x) / np.linalg.norm(delta_x)

In [ ]:
np.linalg.norm(d3E_w[0:fd_Hw.size] - fd_Hw) / np.linalg.norm(d3E_w)

In [ ]:
ws = optimizer.committedWorkingSet

In [ ]:
np.linalg.norm(ws.getFreeComponent(d3E_w[0:fd_Hw.size] - fd_Hw)) / np.linalg.norm(ws.getFreeComponent(d3E_w[0:fd_Hw.size]))

In [ ]:
np.linalg.norm(d3E_w[0:fd_Hw.size] - fd_Hw) / np.linalg.norm(d3E_w[0:fd_Hw.size])

In [ ]:
np.linalg.norm(ws.getFreeComponent(fd_Hw[0:fd_Hw.size]) + ws.getFreeComponent(H_delta_w) - ws.getFreeComponent(delta_w_rhs)) / np.linalg.norm(delta_w_rhs)

In [ ]:
np.linalg.norm(ws.getFreeComponent(d3E_w[0:fd_Hw.size]) + ws.getFreeComponent(H_delta_w) - ws.getFreeComponent(delta_w_rhs)) / np.linalg.norm(delta_w_rhs)

In [ ]:
np.linalg.norm(ws.getFreeComponent(d3E_w[0:fd_Hw.size]) + ws.getFreeComponent(H_fd_delta_w) - ws.getFreeComponent(delta_w_rhs)) / np.linalg.norm(delta_w_rhs)

In [ ]:
np.linalg.norm(ws.getFreeComponent(fd_Hw[0:fd_Hw.size]) + ws.getFreeComponent(H_fd_delta_w) - ws.getFreeComponent(fd_w_rhs)) / np.linalg.norm(delta_w_rhs)

In [ ]:
np.linalg.norm(ws.getFreeComponent(d3E_w[0:fd_Hw.size])), np.linalg.norm(ws.getFreeComponent(H_delta_w)), np.linalg.norm(ws.getFreeComponent(fd_w_rhs))

In [ ]:
#set_weight_for_one_type(optimizer, umbrella_optimization.OptEnergyType.Full)
umbrella_optimization_finite_diff.gradient_convergence_plot(optimizer, direction, umbrella_optimization.OptEnergyType.Full, minStepSize=1e-9, maxStepSize=1e-1)

In [ ]:
#set_weight_for_one_type(optimizer, umbrella_optimization.OptEnergyType.Umbrella)
umbrella_optimization_finite_diff.gradient_convergence_plot(optimizer, direction, umbrella_optimization.OptEnergyType.Umbrella, minStepSize=1e-9, maxStepSize=1e-1)

In [ ]:
#set_weight_for_one_type(optimizer, umbrella_optimization.OptEnergyType.Target)
umbrella_optimization_finite_diff.gradient_convergence_plot(optimizer, direction, umbrella_optimization.OptEnergyType.Target, minStepSize=1e-9, maxStepSize=1e-1)

In [ ]:
umbrella_optimization_finite_diff.gradient_convergence_plot(optimizer, direction, umbrella_optimization.OptEnergyType.DeploymentForce, minStepSize=1e-9, maxStepSize=1e-1)

### Hessian Finite Difference Errors

In [ ]:
#set_weight_for_one_type(optimizer, umbrella_optimization.OptEnergyType.Full)
umbrella_optimization_finite_diff.hessian_convergence_plot(optimizer, direction, umbrella_optimization_finite_diff.OptEnergyType.Full, minStepSize=1e-9, maxStepSize=1e1)

In [ ]:
#set_weight_for_one_type(optimizer, umbrella_optimization.OptEnergyType.Umbrella)
error = umbrella_optimization_finite_diff.hessian_convergence_plot(optimizer, direction, umbrella_optimization.OptEnergyType.Umbrella, minStepSize=1e-9, maxStepSize=1e1)

In [ ]:
#set_weight_for_one_type(optimizer, umbrella_optimization.OptEnergyType.Target)
umbrella_optimization_finite_diff.hessian_convergence_plot(optimizer, direction, umbrella_optimization.OptEnergyType.Target, minStepSize=1e-9, maxStepSize=1e1)